In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
import cv2
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Function to load images from a directory
def load_images_from_folder(folder, label):
    if not os.path.exists(folder):
        print(f"Directory {folder} does not exist.")
        return [], []

    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (64, 64))  # Resize image to 64x64
            images.append(img)              # Keep the 2D image format for CNN
            labels.append(label)
        else:
            print(f"Could not read image {img_path}")
    return images, labels

# Load and preprocess the dataset
train_normal_path = '/content/drive/MyDrive/diabetesproj/train/normal'
train_diabetes_path = '/content/drive/MyDrive/diabetesproj/train/diabetes'
test_normal_path = '/content/drive/MyDrive/diabetesproj/test/normal'
test_diabetes_path = '/content/drive/MyDrive/diabetesproj/test/diabetes'

# Load training data
train_normal_images, train_normal_labels = load_images_from_folder(train_normal_path, 0)
train_diabetes_images, train_diabetes_labels = load_images_from_folder(train_diabetes_path, 1)

# Load testing data
test_normal_images, test_normal_labels = load_images_from_folder(test_normal_path, 0)
test_diabetes_images, test_diabetes_labels = load_images_from_folder(test_diabetes_path, 1)

# Combine the training and testing data
X_train = np.array(train_normal_images + train_diabetes_images)
y_train = np.array(train_normal_labels + train_diabetes_labels)
X_test = np.array(test_normal_images + test_diabetes_images)
y_test = np.array(test_normal_labels + test_diabetes_labels)

# Normalize pixel values to be between 0 and 1
X_train = X_train / 255.0
X_test = X_test / 255.0

# Flatten the images for the Random Forest classifier
X_train_flattened = X_train.reshape(len(X_train), -1)
X_test_flattened = X_test.reshape(len(X_test), -1)

# Train a Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_flattened, y_train)

# Evaluate the model on the test set
y_train_pred = rf_model.predict(X_train_flattened)
y_test_pred = rf_model.predict(X_test_flattened)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print("\nTraining Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)

# Function to make a prediction on a single image using Random Forest
def predict_image_rf(image_path, model):
    # Load the image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"Error: Could not read image {image_path}")
        return None

    # Preprocess the image (resize and normalize)
    img = cv2.resize(img, (64, 64))  # Resize to 64x64
    img = img / 255.0                # Normalize pixel values to be between 0 and 1
    img_flattened = img.reshape(1, -1)  # Flatten to 1D

    # Predict the class (0 for normal, 1 for diabetic)
    prediction = model.predict(img_flattened)
    class_index = prediction[0]

    if class_index == 0:
        result = 'Normal'
    else:
        result = 'Diabetic'

    print(f"Prediction: {result}")
    return result

# Example usage: Change the path below to your image path
image_path = '/content/drive/MyDrive/diabetesproj/test/diabetes/10014_left.jpeg'  # Replace with your image path
predict_image_rf(image_path, rf_model)


Mounted at /content/drive

Training Accuracy: 1.0
Testing Accuracy: 0.9977011494252873
Prediction: Diabetic


'Diabetic'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')